### Import data and do a tiny bit of cleaning (on the 'column' column)

In [58]:
import pandas as pd
import numpy as np
input_data = pd.read_csv("Service.csv", skiprows=6, skipfooter=8,
usecols=[0,2,4,5,6,7,8,9,10],
# na_values="-"
)
input_data.rename(columns={"Unnamed: 4": "load_combination","Unnamed: 2": "column","Unnamed: 0": "support", "Fvert\n[kN]":"Fv", "Fmajor\n[kN]":"Fy","Fminor\n[kN]":"Fx","Mmajor\n[kNm]":"My","Mminor\n[kNm]":"Mx","Mtor\n[kNm]":"Mtor"}, inplace=True)
input_data.ffill(inplace=True)

def column_sizes(input):
    """
    Function to re-write the column string in a more readable way
    """
    return input.split('(')[1].split(')')[0]
    
# Apply the column_sizes function to the imported data. 
input_data.column = input_data.column.apply(column_sizes)

# View the input_data
input_data.tail(20)


c:\Users\rmacdonald\OneDrive - Blyth & Blyth Consulting Engineers\Desktop\Work\Work 2\coding\.venv\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,support,column,load_combination,Fv,Fy,Fx,My,Mx,Mtor
1029,SUP K/#269,UKB 762x267x134,3 Dead,3.3,0.1,0.0,0.0,0.0,0.0
1030,SUP K/#269,UKB 762x267x134,4 Services,3.3,0.1,0.0,0.0,0.0,0.0
1031,SUP K/#269,UKB 762x267x134,5 Imposed,8.0,0.3,0.0,0.0,0.0,0.0
1032,SUP K/#269,UKB 762x267x134,6 Snow,9.5,0.3,0.0,0.0,0.0,0.0
1033,SUP K/#269,UKB 762x267x134,"8 Wind 0,Cpi -0.3,+Cpe",-4.9,0.0,5.8,0.0,0.0,0.0
1034,SUP K/#269,UKB 762x267x134,"9 Wind 0,Cpi -0.3,-Cpe",-4.9,0.0,5.8,0.0,0.0,0.0
1035,SUP K/#269,UKB 762x267x134,"10 Wind 0,Cpi 0.2,+Cpe",-13.4,-0.3,20.1,0.0,0.0,0.0
1036,SUP K/#269,UKB 762x267x134,"11 Wind 0,Cpi 0.2,-Cpe",-13.4,-0.3,20.1,0.0,0.0,0.0
1037,SUP K/#269,UKB 762x267x134,"12 Wind 90,Cpi -0.3,+Cpe",18.2,12.6,5.7,0.0,0.0,0.0
1038,SUP K/#269,UKB 762x267x134,"13 Wind 90,Cpi -0.3,-Cpe",9.7,8.8,5.7,0.0,0.0,0.0


### Support Class Definition

In [69]:

# define support class
class Support():
    def __init__(self, df, length, width):
        self.df = df
        self.calculate_initial_length_and_width()        
        # self.optimise_area()

        self.starting_length = self.df

    
    def calculate_initial_length_and_width(self,):
        """
        Uses the data in the df attribute to calculate initial length, width and area values and stores them within the object. 
        """
        assert len(self.df.column.unique()) == 1, print(f"Not all of the column values are the same")

        length = int(self.df.column.iloc[0][3:].split('x')[0])
        width = int(self.df.column.iloc[0][3:].split('x')[1])
        self.length = length
        self.width = width
        self.area = length * width        

    def optimise_area(self):
        length = self.length # initialise length/width for function 
        width = self.width
        area= length * width
        bearing_capacity=150 # make input nicer... ask user for input?
        bearing_pressure = (df.Fv.max())/a
        print(f"Starting bearing pressure {bearing_pressure}, TRYING AREA {area}")
        while bearing_pressure > 150:
            length = length+0.0001
            width = width+0.0001
            area = width * length
            bearing_pressure = (df.Fv.max())/a

        print(f"Calculated bearing pressure {bearing_pressure}, TRYING AREA {area}")

        # can just do force/pressure to get maximum area then work out l, b beginning from there? most important bit is to figure out sliding
        # capacity for each of these, therefore will maybe be based on that
        # where should depth come into it - minimum depth that is okay for bearing pressure and sliding? Bearing pressure will never change, therefore can define that based on maximum FV, then use sliding to define L and B but always has to be less than calculated area, however depth can change

        # store optimised values
        self.length = length
        self.width = width
        self.area = length * width 
        # self.optimised_area=optimised_area



### Create the Support objects - and put them in the 'support_dictionary' for ease of access...

In [70]:
support_dictionary= {}
# New dataframes created for each different support type. 
# These dataframes are then used to create independent Support() objects, with different dataframes.

column_size=input_data.column.unique()

for support_name in column_size:
    independent_support_data=input_data.loc[input_data['column']==support_name]
    support_dictionary[support_name]=Support(df=independent_support_data, length=1000, width = 1000)

print(f"Support dict keys {support_dictionary.keys()}")


Support dict keys dict_keys(['UKB 762x267x134', 'UKB 457x191x67', 'UC 203x203x60', 'UKB 406x178x54'])


In [61]:
test_support = support_dictionary['UKB 762x267x134']
len(test_support.df.column.unique())
# test_support.width

1

### Create a summary table

In [71]:

# Empty df for storing the summary table
summary_table_df=pd.DataFrame()
# Get the largest Fv Fy and Fx and smallest Fv for all of the supports. 
for key, support in support_dictionary.items():
        for i in ['Fv','Fy','Fx']:
            if i == 'Fv':
                support.df.sort_values(by=i, ascending=False, inplace=True)
                summary_table_df = summary_table_df.append(support.df.iloc[0])
                support.df.sort_values(by=i, ascending=True, inplace=True)
                summary_table_df = summary_table_df.append(support.df.iloc[0])
            else:
                support.df.sort_values(by=i, ascending=False, inplace=True, key=abs)
                summary_table_df = summary_table_df.append(support.df.iloc[0])

summary_table_df.head(10)
# # assert len(summary_table_df) == 12, "Ahhhh noooo"
report_df=summary_table_df.copy()
report_df.set_index('column', inplace=True)
report_df.rename({'load_combination':'Load Combination'},axis='columns', inplace=True)
print(report_df.to_latex(columns=['Load Combination','Fv','Fy','Fx'],index_names=False))

\begin{tabular}{llrrr}
\toprule
{} &                 Load Combination &     Fv &    Fy &     Fx \\
\midrule
UKB 762x267x134 &           8 Wind 0,Cpi -0.3,+Cpe &  206.1 &  18.0 &  120.8 \\
UKB 762x267x134 &           10 Wind 0,Cpi 0.2,+Cpe & -326.6 & -29.3 & -118.1 \\
UKB 762x267x134 &          15 Wind 90,Cpi 0.2,-Cpe & -121.3 & -66.8 &    0.0 \\
UKB 762x267x134 &           9 Wind 0,Cpi -0.3,-Cpe &  206.1 &  18.0 &  120.8 \\
UKB 457x191x67  &         12 Wind 90,Cpi -0.3,+Cpe &   39.0 &  22.7 &   -0.1 \\
UKB 457x191x67  &          14 Wind 90,Cpi 0.2,+Cpe &  -43.8 &  33.5 &   -0.2 \\
UKB 457x191x67  &           9 Wind 0,Cpi -0.3,-Cpe &  -11.5 &  56.8 &    0.0 \\
UKB 457x191x67  &         12 Wind 90,Cpi -0.3,+Cpe &  -32.4 &  12.3 &   -0.2 \\
UC 203x203x60   &  1 Self weight - excluding slabs &    9.4 &   0.0 &    0.0 \\
UC 203x203x60   &          14 Wind 90,Cpi 0.2,+Cpe &   -9.9 &  -7.2 &  -13.4 \\
UC 203x203x60   &           11 Wind 0,Cpi 0.2,-Cpe &   -8.1 &  24.1 &   -5.8 \\
UC 203x203x6

c:\Users\rmacdonald\OneDrive - Blyth & Blyth Consulting Engineers\Desktop\Work\Work 2\coding\.venv\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [63]:
from numpy import sin,cos,tan,cosh,sinh,tanh,deg2rad,sqrt

# density of concrete (kN/m3)
P_CONCRETE=23.6
# depth of pad footing (mm)
H=750

A=l*w






# depth of soil over pad (mm)
H_SOIL=500 #mm
P_SOIL=20 #mm
SHEAR_STRENGTH=deg2rad(25) #rad
BASE_FRICTION=deg2rad(19.3) #rad
P_BEARING=150 #kn/m2

# column details
I_A=850 #mm base length
B_A=450 #mm base width
E_PXA=0 #mm eccentrincity in x
E_PYA=0 #mm ecc in y

# loading
# P=df.Fv #(kN), axial 
P=4
H_Y=1 #df.FY
H_X=1 #df.Fv
M_X=0
M_Y=0

# Dead surcharge load (kN/m2)
F_Gsur=0 
F_Qsur=0

# Pad footing self weight
F_SWT=H*P_CONCRETE
# soil self weight
F_SOIL=H_SOIL*P_SOIL
# total foundation load
F=A*(F_SOIL+F_Qsur+F_SOIL+F_Gsur)



# Stability against sliding (kN)
H_friction=max((P+(F_Gsur+F_Qsur+F_SWT),0)) #resistance due to base friction
K_P=(1+sin(SHEAR_STRENGTH))/(1-sin(SHEAR_STRENGTH)) #passive pressure coef
H_D=sqrt(H_Y^2+H_X^2)
ALPHA=A*tan(H_Y/H_X)
H_xpas=0.5*K_P*(H^2+2*H*H_SOIL)*width*P_SOIL
H_ypas=0.5*K_P*(H^2+2*H*H_SOIL)*length*P_SOIL
# resultant passive resistance
H_RESULTANT=abs(H_xpas*cos(ALPHA))+abs(H_ypas*sin(ALPHA))
H_RES=H_friction+H_RESULTANT #should be greater than resultant horizontal load then PASS


# STABILITY AGAINST TURNING
M_XOT=M_X+H_X*H
M_XSUR=A*(F_Gsur+F_SWT+F_SOIL)*length/2
M_XAXIAL=P*(length/2-E_PXA)
M_XRES=M_XSUR+M_XAXIAL

M_YOT=M_Y+H_Y*H
M_YSUR=A*(F_Gsur+F_SWT+F_SOIL)*width/2
M_YAXIAL=P*(width/2-E_PYA)
M_YRES=M_YSUR+M_YAXIAL

# Pad base reactions
T=F+P
E_TX=(P*E_PXA+M_X+H_X*H)/T
E_TY=(P*E_PYA+M_Y+H_Y*H)/T


Q_1=T/A-6*T*E_TX/(length*A)-6*T*E_TY/(width*A)
Q_2=T/A-6*T*E_TX/(length*A)+6*T*E_TY/(width*A)
Q_3=T/A+6*T*E_TX/(length*A)-6*T*E_TY/(width*A)
Q_4=T/A+6*T*E_TX/(length*A)+6*T*E_TY/(width*A)

Q_MIN=min(Q_1,Q_2,Q_3,Q_4)
Q_MAX=max(Q_1,Q_2,Q_3,Q_4) #Is maximum base pressure less than the allowable



# initial guess for length and width based upon A = P/FV,MAX



NameError: name 'l' is not defined